In [4]:
import os
import sys
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
from util import PipelineManager
from util import DesignFileLoader

#s3 addresses for input files and output files
s3_input_files_address = "s3://ucsd-ccbb-interns/Mustafa/chip_test/"
s3_output_files_address = "s3://ucsd-ccbb-interns/Mustafa/chip_test/"

## CFNCluster name
your_cluster_name = "mustafa7"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/keys/interns_oregon_key.pem"

## Project information
project_name = "histone_output_check"

#options: homer
workflow = "homer"

#options: hg19, mm10
genome = "hg19"

#options: factor, histone
style = "histone"

## If delete cfncluster after job is done.
delete_cfncluster= False


##order does not matter

##can be fastqc, trim, bowtie, multiqc, make_tag_directory, make_UCSC_file, 
#find_peaks, annotate_peaks, pos2bed, find_motifs_genome
analysis_steps = {
                    "fastqc"
                    ,"trim"
                    ,"bowtie"
#                     ,"multiqc"
                    ,"make_tag_directory"
                    ,"make_UCSC_file"
                    ,"find_peaks"
                    ,"annotate_peaks"
                    ,"pos2bed"
                    ,"find_motifs_genome"
                }

#path to chipseq design file
#examples in cirrus_root/data/cirrus-ngs/
design_file = "/home/mustafa/ccbb/cirrus-ngs/data/cirrus-ngs/testing_chip.txt"

print("variables set")

variables set


In [5]:
## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa7 does exist.
Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="34.214.180.149"
Output:"MasterPrivateIP"="172.31.33.31"
Output:"GangliaPublicURL"="http://34.214.180.149/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.33.31/ganglia/"

connecting
connected


In [6]:
sample_list, group_list, pairs_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("ChiPSeq", ssh_client, project_name, workflow, analysis_steps, s3_input_files_address,
                       sample_list, group_list, s3_output_files_address, genome, style, pairs_list)

[['chipseq_sample1_chip.fastq.gz'], ['chipseq_sample1_input.fastq.gz'], ['chipseq_sample2_chip.fastq.gz'], ['chipseq_sample2_input.fastq.gz']]
['groupA', 'groupA', 'groupB', 'groupB']
{'chipseq_sample1_chip': 'chipseq_sample1_input', 'chipseq_sample2_chip': 'chipseq_sample2_input'}
making the yaml file...
copying yaml file to remote master node...
histone_output_check.yaml
/shared/workspace/Pipelines/yaml_files/ChiPSeq/homer
executing pipeline...
Executing nohup bash /shared/workspace/Pipelines/scripts/run.sh /shared/workspace/Pipelines/yaml_files/ChiPSeq/homer/histone_output_check.yaml /shared/workspace/logs/ChiPSeq/homer/histone_output_check ChiPSeq_homer


In [ ]:
step = "all" #can be any step in analysis_steps or "all"
PipelineManager.check_status(ssh_client, step, "ChiPSeq", workflow, project_name, analysis_steps,verbose=True)